In [ ]:
# import os
# import requests
# import json

# # Retrieve the API key from the environment
# api_key = os.getenv("DSBA_LLAMA3_KEY")

# API_URL = "https://lasyaed--vllm-openai-compatible-serve.modal.run/rag-query"  # Replace with your actual URL

# def test_rag_query(query):
#     headers = {
#         "Content-Type": "application/json",
#         "Authorization": f"Bearer {api_key}"  # Use the API key from environment variable
#     }
#     data = json.dumps({"query": query})

#     try:
#         response = requests.post(API_URL, headers=headers, data=data)
#         print(f"Status Code: {response.status_code}")
#         if response.status_code == 200:
#             print("Response:", response.json())
#         else:
#             print("Error Message:", response.text)
#     except Exception as e:
#         print(f"Failed to communicate with the server: {e}")

# test_rag_query("Explain METTLER TOLEDO's intellectual property policy.")


Status Code: 200
Response: {'response': 'Error generating response.'}


In [ ]:
!pip install faiss-cpu
!pip install openai==0.28.0
!pip install python-dotenv
!pip install sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 2.3 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.52.2
    Uninstalling openai-1.52.2:
      Successfully uninstalled openai-1.52.2


In [ ]:
import faiss
import openai
import os
from sentence_transformers import SentenceTransformer
from google.colab import userdata

# Load the embedding model
embedding_model_name = "all-MiniLM-L6-v2"  # Example embedding model
embedding_model = SentenceTransformer(embedding_model_name)

# Set your OpenAI API key
openai.api_key = userdata.get('OPENAI_API_KEY')

# Load FAISS index
index_path = "faiss_index.bin"
faiss_index = faiss.read_index(index_path)

# Load filename_mapping.txt for metadata
def load_filename_mapping(file_path):
    context_dict = {}
    with open(file_path, "r", encoding="utf-8") as f:
        for line in f:
            parts = line.strip().split("\t")
            if len(parts) == 5:  # Assumes structure: "idx\tdoc_id\tsection_title\tsubsection\tfile_name"
                idx, doc_id, section_title, subsection, file_name = parts
                context_dict[int(idx)] = {
                    "Document ID": doc_id,
                    "Section Title": section_title,
                    "Subsection": subsection,
                    "File Name": file_name
                }
    return context_dict

# Create combined context by appending document content with metadata
def create_combined_context(context_dict, indices):
    combined_context = ""
    for idx in indices:
        entry = context_dict.get(idx)
        if entry:
            file_path = f"data/{entry['File Name']}"  # Assumes documents are stored in 'data/' directory
            try:
                with open(file_path, "r", encoding="utf-8") as f:
                    content = f.read()
                # Append content with metadata for clarity
                combined_context += (
                    f"Document ID: {entry['Document ID']}\n"
                    f"Section Title: {entry['Section Title']}\n"
                    f"Subsection: {entry['Subsection']}\n"
                    f"Content:\n{content}\n\n"
                )
            except FileNotFoundError:
                print(f"Warning: {file_path} not found.")
    return combined_context

# Function to embed query, retrieve similar documents, and generate response with ChatGPT
def generate_response_with_context(query, top_k=2, model="gpt-3.5-turbo", temperature=0.7):
    # Embed the query
    query_embedding = embedding_model.encode(query).reshape(1, -1)

    # Retrieve similar documents using FAISS
    distances, indices = faiss_index.search(query_embedding, top_k)
    print("Retrieved document indices:", indices[0])  # Display indices of retrieved documents

    # Load filename_mapping.txt and retrieve combined context for top_k documents
    filename_mapping_path = "filename_mapping.txt"
    context_dict = load_filename_mapping(filename_mapping_path)
    combined_context = create_combined_context(context_dict, indices[0])

    # Create messages for the ChatCompletion call
    messages = [
        {"role": "system", "content": "Use the provided information to answer the question."},
        {"role": "user", "content": f"Context:\n{combined_context}\n\nQuestion:\n{query}"}
    ]

    # Call OpenAI API with the combined context
    try:
        response = openai.ChatCompletion.create(
            model=model,
            messages=messages,
            max_tokens=200,
            temperature=temperature
        )
        return response.choices[0].message['content'].strip()  # Return the generated response text
    except Exception as e:
        print(f"Error calling OpenAI API: {e}")
        return None



In [ ]:
# Test the function with a sample query
sample_query = "Does METTLER TOLEDO's waive any provision?"
response = generate_response_with_context(sample_query)
print(response)


Retrieved document indices: [316 307]
No, METTLER TOLEDO does not waive any provision.


In [ ]:
sample_query = "Does METTLER TOLEDO provide products and services in all countries?"
response = generate_response_with_context(sample_query)
print(response)

Retrieved document indices: [295 282]
No, METTLER TOLEDO does not provide products and services in all countries. The company operates in various countries worldwide, but its product availability and service offerings may vary depending on the region.


In [ ]:
sample_query = "how to submit a contest entry for BurgerKing?"
response = generate_response_with_context(sample_query)
print(response)

Retrieved document indices: [85 82]
To submit a contest entry for Burger King, you typically need to follow the guidelines and requirements set by the specific contest. This can usually be found on the Burger King website or on their social media channels where the contest is being promoted. Here are some general steps you might need to take:

1. Visit the Burger King website or their official social media pages to find the contest details.
2. Read the rules and regulations of the contest carefully to understand the requirements for the entry.
3. Create your contest entry according to the guidelines provided. This could involve submitting a photo, video, written content, or any other form of entry specified by the contest rules.
4. Submit your entry through the designated method, which could be through the contest website, social media platform, or via email.
5. Make sure to meet the deadline for submission and adhere to any other instructions provided.

Remember, the specific steps ma

In [ ]:
sample_query = "What personal information does Bank of America access from my wireless account to prevent fraud?"
response = generate_response_with_context(sample_query)
print(response)

Retrieved document indices: [269 271]
Bank of America accesses the following personal information from your wireless account to prevent fraud:

1. Location information
2. Phone number
3. Device information


In [ ]:
sample_query = "What information might my wireless operator disclose to Bank of America or Zelle® when I use their service?"
response = generate_response_with_context(sample_query)
print(response)

Retrieved document indices: [269 184]
When you use a wireless operator's service to make transactions through Bank of America or Zelle®, the wireless operator may disclose information such as your phone number, location data, device information, and details of the transactions to facilitate the service and ensure its security. This information is typically shared in accordance with the wireless operator's privacy policy and in compliance with relevant laws and regulations.


In [ ]:
sample_query = "Are third-party delivery services covered by the Chick-fil-A One rewards program?"
response = generate_response_with_context(sample_query)
print(response)

Retrieved document indices: [ 71 345]
Yes, Chick-fil-A One rewards program covers orders placed through third-party delivery services as long as the account is linked to the delivery platform.


In [ ]:
sample_query = "Does Chick-fil-A's Terms and Conditions allow class action lawsuits?"
response = generate_response_with_context(sample_query)
print(response)

Retrieved document indices: [ 33 374]
Chick-fil-A's Terms and Conditions do not allow class action lawsuits.


In [ ]:
sample_query = "Can I redeem my Chick-fil-A One points at all locations?"
response = generate_response_with_context(sample_query)
print(response)

Retrieved document indices: [23 74]
Yes, you can redeem your Chick-fil-A One points at all participating Chick-fil-A locations.


In [ ]:
sample_query = "What is Chick-fil-A's policy on gift offers and digital rewards?"
response = generate_response_with_context(sample_query)
print(response)

Retrieved document indices: [347 350]
Chick-fil-A has a policy that allows customers to earn free food through their Chick-fil-A One rewards program. Customers can earn points for each purchase made, and once a certain number of points is reached, they can redeem them for free food items. Chick-fil-A also occasionally offers digital rewards and promotions to their customers through the Chick-fil-A One app. Customers can check the app for special offers and promotions available to them.


In [ ]:
sample_query = "What are OpenAI's restrictions on using generated output for commercial purposes?"
response = generate_response_with_context(sample_query)
print(response)

Retrieved document indices: [12 16]
OpenAI's GPT-3 API terms of use state that users are not allowed to use the generated output for commercial purposes unless they have received written approval from OpenAI. This restriction is in place to prevent misuse of the technology for potential harm or unethical purposes.


In [ ]:
sample_query = "Can I use ChatGPT's output to develop competing AI models?"
response = generate_response_with_context(sample_query)
print(response)

Retrieved document indices: [107 351]
No, you cannot use ChatGPT's output to develop competing AI models. The output generated by ChatGPT is protected by copyright laws and is intended for personal, non-commercial use only. Developing competing AI models using ChatGPT's output would be a violation of the terms of service and copyright laws.


In [ ]:
sample_query = "Under what conditions would I be required to seek an additional license from Meta if I intend to commercially distribute products utilizing Llama 2 outputs?"
response = generate_response_with_context(sample_query)
print(response)

Retrieved document indices: [121 123]
If you intend to commercially distribute products utilizing Llama 2 outputs, you would be required to seek an additional license from Meta if you plan to include any proprietary technology or intellectual property owned by Meta in your products. This is because using Meta's proprietary technology or intellectual property for commercial purposes may require a separate license agreement to ensure compliance with Meta's terms and conditions. It is important to review Meta's licensing policies and agreements to understand your obligations and requirements when incorporating Llama 2 outputs into your commercial products.
